# Fine-tuning FunctionGemma for Alpha-MCP

This notebook trains **Google FunctionGemma 270M** on the **Alpha-MCP** toolset.
It is **checkpoint-aware** — you can stop and resume training at any point,
and re-run with expanded datasets when `alpha-mcp` adds new tools.

## Architecture
- **Base Model**: `google/functiongemma-270m-it` (gated — must login)
- **Method**: LoRA + SFT via `trl`
- **Dataset**: `data/synthetic_training.jsonl` (1200+ synthetic tool-call examples)
- **Format**: FunctionGemma `<start_function_call>call:name{...}<end_function_call>`
- **Hardware**: GTX 1060 6GB compatible (fp16, batch=1)

## Checkpoint Resume
Set `RESUME_FROM_CHECKPOINT = True` and point `CHECKPOINT_DIR` at your latest checkpoint
folder (e.g. `checkpoints/checkpoint-500`) to continue a previous run.

In [ ]:
import torch
import json
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from huggingface_hub import login
import time

print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()} | Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB' if torch.cuda.is_available() else '')

## 0. Config — Edit Here

In [ ]:
# ── Model ─────────────────────────────────────────────────────────────────────
MODEL_NAME   = 'google/functiongemma-270m-it'  # FunctionGemma 270M (gated)
# MODEL_NAME = 'google/gemma-2-2b-it'           # Uncomment for Gemma 2 2B (better reasoning)

# ── Dataset ───────────────────────────────────────────────────────────────────
DATASET_PATH  = 'data/synthetic_training.jsonl'

# ── Output & Checkpoints ──────────────────────────────────────────────────────
OUTPUT_DIR    = 'outputs/alpha_functiongemma'
CHECKPOINT_DIR = 'checkpoints'  # Checkpoints saved here every N steps

# ── Resume Training ───────────────────────────────────────────────────────────
RESUME_FROM_CHECKPOINT = False  # Set True to resume from latest checkpoint

# ── LoRA Config ───────────────────────────────────────────────────────────────
LORA_R         = 16
LORA_ALPHA     = 16
LORA_DROPOUT   = 0.05

# ── Training ──────────────────────────────────────────────────────────────────
MAX_SEQ_LENGTH = 512
BATCH_SIZE     = 1
GRAD_ACCUM     = 8
LEARNING_RATE  = 2e-4
NUM_EPOCHS     = 3         # More epochs = better tool discrimination
SAVE_STEPS     = 100       # Checkpoint every 100 steps
WARMUP_STEPS   = 30

print('✅ Config loaded.')
print(f'   Model:    {MODEL_NAME}')
print(f'   Dataset:  {DATASET_PATH}')
print(f'   Resume:   {RESUME_FROM_CHECKPOINT}')
print(f'   Epochs:   {NUM_EPOCHS} | Save every: {SAVE_STEPS} steps')

## 1. HuggingFace Login (Required for Gated Model)

In [ ]:
# Accept the license at: https://huggingface.co/google/functiongemma-270m-it
# Then get your token at: https://huggingface.co/settings/tokens
login()

## 2. Generate Synthetic Training Data

This runs the data generator. **Re-run this cell whenever alpha-mcp adds new tools.**
The generator will produce a fresh dataset reflecting the new schemas.

In [ ]:
import subprocess
result = subprocess.run(
    ['python', 'scripts/generate_data.py', '--n', '1200'],
    capture_output=True, text=True
)
print(result.stdout)
if result.returncode != 0:
    print('❌ Error:', result.stderr)

## 3. Load Model & Tokenizer

In [ ]:
print(f'⏳ Loading {MODEL_NAME}...')
t0 = time.time()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print(f'✅ Loaded in {time.time()-t0:.1f}s')
print(f'   VRAM after load: {torch.cuda.memory_allocated()/1024**3:.2f} GB')

## 4. Pre-Training Baseline (Tool Call Quality)

In [ ]:
TEST_PROMPTS = [
    'What are the best Diamond setups in the S&P 500 right now?',
    'Give me a full technical breakdown on NVDA.',
    'Is the market in bull or bear mode?',
    'Run a risk audit on my positions: AAPL, TSLA, MSFT.',
    'Diamond screen the NASDAQ 100.',
]

print('=== PRE-TRAINING BASELINE ===')
model.eval()
for prompt in TEST_PROMPTS:
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(out[0], skip_special_tokens=False)
    print(f'\n🔹 {prompt}')
    print(f'🔸 {resp.strip()}')
    print('-'*50)
model.train()

## 5. Apply LoRA Adapters

In [ ]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj',
                    'gate_proj', 'up_proj', 'down_proj'],
    lora_dropout=LORA_DROPOUT,
    bias='none',
    task_type='CAUSAL_LM',
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 6. Prepare Dataset

Formats each conversation into the FunctionGemma `<start_of_turn>` template.

In [ ]:
raw_dataset = load_dataset('json', data_files=DATASET_PATH, split='train')
print(f'✅ Loaded {len(raw_dataset)} training examples')

def format_conversation(example):
    """Convert ShareGPT format → FunctionGemma turn format."""
    text = ''
    for turn in example['conversations']:
        role = turn['role']
        content = turn['content']
        text += f'<start_of_turn>{role}\n{content}<end_of_turn>\n'
    text += tokenizer.eos_token
    return {'text': text}

dataset = raw_dataset.map(format_conversation, batched=False)

# Quick sample inspection
print('\n--- Sample 0 ---')
print(dataset[0]['text'])
print('--- Sample 1 ---')
print(dataset[1]['text'])

## 7. Train (Checkpoint-Aware)

Training saves checkpoints every `SAVE_STEPS` steps.
Set `RESUME_FROM_CHECKPOINT = True` to resume from the latest.

In [ ]:
# Detect latest checkpoint for resume
resume_checkpoint = None
if RESUME_FROM_CHECKPOINT:
    checkpoints = sorted([
        os.path.join(CHECKPOINT_DIR, d) for d in os.listdir(CHECKPOINT_DIR)
        if d.startswith('checkpoint-')
    ], key=lambda x: int(x.split('-')[-1]))
    if checkpoints:
        resume_checkpoint = checkpoints[-1]
        print(f'▶️  Resuming from: {resume_checkpoint}')
    else:
        print('⚠️  No checkpoints found, starting fresh.')

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        dataset_text_field='text',
        max_length=MAX_SEQ_LENGTH,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        bf16=False,
        logging_steps=10,
        optim='adamw_torch',
        weight_decay=0.01,
        lr_scheduler_type='cosine',
        seed=3407,
        output_dir=CHECKPOINT_DIR,  # Checkpoints go here
        save_strategy='steps',
        save_steps=SAVE_STEPS,
        save_total_limit=5,          # Keep last 5 checkpoints
        load_best_model_at_end=False,
        report_to='none',
    ),
)

print(f'🚀 Starting training ({NUM_EPOCHS} epochs, {len(dataset)} samples, resume={RESUME_FROM_CHECKPOINT})')
t0 = time.time()
trainer.train(resume_from_checkpoint=resume_checkpoint)
print(f'✅ Training complete in {(time.time()-t0)/60:.1f} min')

## 8. Post-Training Verification

In [ ]:
print('=== POST-TRAINING TOOL CALL QUALITY ===')
model.eval()
for prompt in TEST_PROMPTS:
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(out[0], skip_special_tokens=False)
    has_call = '<start_function_call>' in resp
    status = '✅' if has_call else '❌'
    print(f'{status} {prompt}')
    if has_call:
        call_part = resp.split('<start_function_call>')[1].split('<end_function_call>')[0]
        print(f'   → {call_part}')
    print()

## 9. Save Final Model

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f'🏁 Final adapter saved to: {OUTPUT_DIR}')

# Save training metadata for reproducibility
import json
meta = {
    'base_model': MODEL_NAME,
    'dataset': DATASET_PATH,
    'num_samples': len(dataset),
    'epochs': NUM_EPOCHS,
    'lora_r': LORA_R,
    'lora_alpha': LORA_ALPHA,
    'timestamp': time.strftime('%Y-%m-%dT%H:%M:%S'),
    'tools': ['analyze_ticker_full', 'run_diamond_screen', 'get_market_pulse', 'run_risk_audit'],
}
with open(f'{OUTPUT_DIR}/training_metadata.json', 'w') as f:
    json.dump(meta, f, indent=2)
print(f'Metadata: {OUTPUT_DIR}/training_metadata.json')